In [4]:
import numpy as np
import pandas as pd


## EDA

In [5]:
#df = pd.read_csv('data/jobs.csv')

In [4]:
"""
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")
print("-"*100)
print(f"Number of missing values: {df.isnull().sum().sum()}")
print("-"*100)
print(f"Number of duplicated rows: {df.duplicated().sum()}")
print("-"*100)
"""

'\nprint(f"Number of rows: {df.shape[0]}")\nprint(f"Number of columns: {df.shape[1]}")\nprint("-"*100)\nprint(f"Number of missing values: {df.isnull().sum().sum()}")\nprint("-"*100)\nprint(f"Number of duplicated rows: {df.duplicated().sum()}")\nprint("-"*100)\n'

In [5]:
# drop missing values and reset index
#df_clean = df.dropna().reset_index(drop=True)   

# drop id column
#df_clean = df_clean.drop(columns=['lid'])


In [6]:
# save the cleaned data to a new csv file
#df_clean.to_csv('data/jobs_clean.csv', index=False)

In [6]:
df = pd.read_csv('data/jobs_clean.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97417 entries, 0 to 97416
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   jobTitle           97417 non-null  object
 1   companyName        97417 non-null  object
 2   jobDescRaw         97417 non-null  object
 3   finalZipcode       97417 non-null  object
 4   finalState         97417 non-null  object
 5   finalCity          97417 non-null  object
 6   companyBranchName  97417 non-null  object
 7   jobDescUrl         97417 non-null  object
 8   nlpBenefits        97417 non-null  object
 9   nlpSkills          97417 non-null  object
 10  nlpSoftSkills      97417 non-null  object
 11  nlpDegreeLevel     97417 non-null  object
 12  nlpEmployment      97417 non-null  object
 13  nlpSeniority       97417 non-null  object
 14  correctDate        97417 non-null  object
 15  scrapedLocation    97417 non-null  object
dtypes: object(16)
memory usage: 11.9+ MB


In [7]:
pd.set_option('display.max_columns', None)  

In [8]:
# print out all the columns of the first row
for i in range(len(df.columns)):
    print(f"{df.columns[i]}: {df.iloc[0, i]}")

jobTitle: Nuclear Medicine Tech, Full Time, Day Shift
companyName: Adventist Health
jobDescRaw: <div class="normalText"><p>Job Description</p>
<p style="height:6px;"></p>
<p>LOCATION SUMMARY:</p>
<p>Located in Templeton amongst the oak trees, Adventist Health Twin Cities has been serving northern San Luis Obispo County since 1977. Comprised of a 122-bed acute care facility, our team provides exceptional care in emergency medicine, orthopedics, obstetrics, digestive disorders, wound care, and various medical, surgical and outpatient services. Locals enjoy weekly farmers markets in downtown Templeton, farm-to-fork dining, beautiful landscapes for hiking and biking, and beach days just fifteen minutes away at Cambria and Morro Bay.</p>
<p style="height:6px;"></p>
<p>POSITION SUMMARY:</p>
<p>This role is represented by the SEIU. Responsible for the Nuclear Imaging of the newborn, pediatric, adolescent, adult and geriatric patient as ordered by the physician post administration of a radioac

In [9]:
df.drop(columns=['correctDate'], inplace=True)

In [10]:
list_cols = ['nlpBenefits', 'nlpSkills', 'nlpSoftSkills', 'nlpDegreeLevel']
for col in list_cols:
    df[col] = df[col].apply(lambda x: eval(x) if isinstance(x, str) and x.startswith('[') else x)
    # Convert empty lists to [None] in specified columns
    df[col] = df[col].apply(lambda x: [None] if isinstance(x, list) and len(x) == 0 else x)
    # remove duplicates within the list
    df[col] = df[col].apply(lambda x: list(set(x)) if isinstance(x, list) else x)
    # convert list into string with join
    df[col] = df[col].apply(lambda x: ' '.join(str(s) for s in x) if isinstance(x, list) else x)


In [11]:
df['nlpBenefits'][1]

'Hsa/Fsa Vision Insurance Dental Insurance Eap Paid Time Off Health Insurance 401K Life Insurance'

In [13]:
pip install bs4

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 186 kB 4.0 MB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [14]:
from bs4 import BeautifulSoup

# utilze the function to clean the html text in the jobDescRaw column
def clean_html_text(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text(separator=' ', strip=True)

df['jobDesc'] = df['jobDescRaw'].apply(clean_html_text)

In [15]:
df.drop(columns=['jobDescRaw'], inplace=True)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97417 entries, 0 to 97416
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   jobTitle           97417 non-null  object
 1   companyName        97417 non-null  object
 2   finalZipcode       97417 non-null  object
 3   finalState         97417 non-null  object
 4   finalCity          97417 non-null  object
 5   companyBranchName  97417 non-null  object
 6   jobDescUrl         97417 non-null  object
 7   nlpBenefits        97417 non-null  object
 8   nlpSkills          97417 non-null  object
 9   nlpSoftSkills      97417 non-null  object
 10  nlpDegreeLevel     97417 non-null  object
 11  nlpEmployment      97417 non-null  object
 12  nlpSeniority       97417 non-null  object
 13  scrapedLocation    97417 non-null  object
 14  jobDesc            97417 non-null  object
dtypes: object(15)
memory usage: 11.1+ MB


In [17]:
# access the basic information of each column
for col in df.columns:
    print(df[col].describe())


count               97417
unique              49871
top       Delivery Driver
freq                  733
Name: jobTitle, dtype: object
count        97417
unique       24460
top       DoorDash
freq          4438
Name: companyName, dtype: object
count      97417
unique     17364
top       remote
freq         632
Name: finalZipcode, dtype: object
count     97417
unique      178
top          CA
freq       5126
Name: finalState, dtype: object
count        97417
unique        6995
top       New York
freq           751
Name: finalCity, dtype: object
count                                       97417
unique                                      70249
top       Barcadia Bar & Grill || New Orleans, LA
freq                                          139
Name: companyBranchName, dtype: object
count                                                 97417
unique                                                97417
top       https://www.postjobfree.com/job/vubjwa/nuclear...
freq                             

There are several informations we can get from this dataset:
- DoorDash being the top employer and Delivery Driver being the top job title suggests a strong presence of gig/delivery jobs in the dataset.
- "Remote" appearing as the top zipcode indicates the dataset likely includes a significant number of remote positions.
- California, New York, and Chicago appear to be hotspots for job postings.
- A substantial number of entries lack specified benefits, skills, and education requirements (as indicated by "None" values).
- The dataset is heavily skewed toward full-time and entry-level positions.

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97417 entries, 0 to 97416
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   jobTitle           97417 non-null  object
 1   companyName        97417 non-null  object
 2   finalZipcode       97417 non-null  object
 3   finalState         97417 non-null  object
 4   finalCity          97417 non-null  object
 5   companyBranchName  97417 non-null  object
 6   jobDescUrl         97417 non-null  object
 7   nlpBenefits        97417 non-null  object
 8   nlpSkills          97417 non-null  object
 9   nlpSoftSkills      97417 non-null  object
 10  nlpDegreeLevel     97417 non-null  object
 11  nlpEmployment      97417 non-null  object
 12  nlpSeniority       97417 non-null  object
 13  scrapedLocation    97417 non-null  object
 14  jobDesc            97417 non-null  object
dtypes: object(15)
memory usage: 11.1+ MB


In [18]:
df.finalZipcode.value_counts()

remote    632
80012     108
89119      83
67232      81
30604      80
         ... 
85709       1
61485       1
98113       1
32859       1
54927       1
Name: finalZipcode, Length: 17364, dtype: int64

In [19]:
df.nlpDegreeLevel.value_counts()

None                                                                               39535
Certificate                                                                        12235
Bachelors                                                                           5880
High School                                                                         5729
Associates                                                                          4774
                                                                                   ...  
Masters Bachelors Doctoral Associates Postmasters Certificate Certificate              1
Bachelors Associates Postmasters Certificate                                           1
Associates Unknown Masters Doctoral                                                    1
Masters Bachelors Associates Doctoral Postbaccalaureate Certificate Certificate        1
Doctoral Unknown Certificate Masters                                                   1
Name: nlpDegreeLevel,

In [19]:
df.jobDesc.value_counts()

jobDesc
Local CDL-A Truck Drivers Wanted for Dedicated Account. Home Daily $1,500 Minimum Weekly Guarantee Newer Trucks Full Benefits Apply Now                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [20]:
import re
def clean_text(text):
  if not isinstance(text, str):
    return ""
  text = text.lower()
  text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
  text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
  text = re.sub(r'<.*?>', ' ', text)
  text = re.sub(r'\s+', ' ', text).strip()
  return text

In [24]:
df['jobDesc_clean'] = df['jobDesc'].apply(clean_text)

In [26]:
df['jobDesc_clean'][0]

'job description location summary located in templeton amongst the oak trees adventist health twin cities has been serving northern san luis obispo county since 1977 comprised of a 122bed acute care facility our team provides exceptional care in emergency medicine orthopedics obstetrics digestive disorders wound care and various medical surgical and outpatient services locals enjoy weekly farmers markets in downtown templeton farmtofork dining beautiful landscapes for hiking and biking and beach days just fifteen minutes away at cambria and morro bay position summary this role is represented by the seiu responsible for the nuclear imaging of the newborn pediatric adolescent adult and geriatric patient as ordered by the physician post administration of a radioactive material positions and instructs the patient for nuclear medicine examinations adjusts the imaging equipment determines proper dosage and desired exposure time for each image following alara guidelines job requirements educa

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97417 entries, 0 to 97416
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   jobTitle           97417 non-null  object
 1   companyName        97417 non-null  object
 2   finalZipcode       97417 non-null  object
 3   finalState         97417 non-null  object
 4   finalCity          97417 non-null  object
 5   companyBranchName  97417 non-null  object
 6   jobDescUrl         97417 non-null  object
 7   nlpBenefits        97417 non-null  object
 8   nlpSkills          97417 non-null  object
 9   nlpSoftSkills      97417 non-null  object
 10  nlpDegreeLevel     97417 non-null  object
 11  nlpEmployment      97417 non-null  object
 12  nlpSeniority       97417 non-null  object
 13  scrapedLocation    97417 non-null  object
 14  jobDesc            97417 non-null  object
 15  jobDesc_clean      97417 non-null  object
dtypes: object(16)
memory usage: 11.9+ MB


In [29]:
df.to_csv('data/jobs_embed.csv', index=False)